# 01) Single-omics EDA
## 실습 목표
1) mRNA·Clinical·Methylation 데이터 로드 & 공통 샘플 정렬
2) 임상/발현 분포와 패턴 시각화
3) 메틸화 EDA (Exploratory Data Analysis)
4) DMP (Differential Methylated Probes) 분석 및 시각화

## 패키지 설치 안내
필요한 경우 아래 명령어를 사용하여 패키지를 설치하세요:

```bash
pip install numpy pandas matplotlib seaborn scikit-learn scipy
```

## 필수 라이브러리 임포트
데이터 처리, 시각화, 통계 분석을 위한 핵심 라이브러리들을 로드합니다.

In [ ]:
import os, numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
import random
from sklearn.decomposition import PCA
from sklearn.manifold import MDS
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Codespace 작업 디렉토리 설정
os.chdir('/workspaces/pnu/practice_omics')

# 한글 폰트 설정 (시스템에 따라 자동 설정)
import platform
system = platform.system()
if system == 'Darwin':  # macOS
    plt.rc('font', family='AppleGothic')
elif system == 'Windows':
    plt.rc('font', family='Malgun Gothic')
else:  # Linux
    import matplotlib.font_manager as fm
    font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
    if os.path.exists(font_path):
        font_prop = fm.FontProperties(fname=font_path)
        plt.rc('font', family=font_prop.get_name())
    else:
        # DejaVu Sans를 fallback으로 사용
        plt.rc('font', family='DejaVu Sans')

# 마이너스 기호 깨짐 방지
plt.rc('axes', unicode_minus=False)

# 시각화 스타일 설정
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (10,6)
np.random.seed(42)

print("Single-omics EDA 분석 시작!")
print("=" * 50)

## 데이터 경로 및 컬럼명 정의
분석에 사용할 파일 경로와 임상 데이터의 주요 컬럼명을 설정합니다.

In [ ]:
# 파일 경로
MRNA = "mrna_matched.txt"
CLIN = "clinical_matched.tsv"
METH = "methylation_matched.txt"             

# 임상 컬럼명
SAMPLE_COL   = "Sample ID"
STATUS_COL   = "Overall Survival Status"      # "1:DECEASED","0:LIVING"
DURATION_COL = "Overall Survival (Months)"
SUBTYPE_COL  = "Subtype"
STAGE_COL    = "Neoplasm Disease Stage American Joint Committee on Cancer Code"
AGE_COL      = "Diagnosis Age"

print("파일 경로 및 컬럼명 설정 완료")

## 데이터 로드 및 전처리
세 가지 오믹스 데이터(mRNA, Clinical, Methylation)를 로드하고 공통 샘플로 정렬합니다.
메틸화 데이터의 경우 형식에 따라 유연하게 처리합니다.

In [ ]:
# 데이터 로드
mrna = pd.read_csv(MRNA, sep="\t", index_col=0)
clinical = pd.read_csv(CLIN, sep="\t")
methyl = pd.read_csv(METH, sep="\t", index_col=0)

# 공통 샘플 정렬
common = sorted(set(mrna.columns) & set(clinical[SAMPLE_COL]) & set(methyl.columns))

mrna = mrna[common]
clinical = clinical[clinical[SAMPLE_COL].isin(common)].reset_index(drop=True)
methyl = methyl[common]

mrna_num = mrna.select_dtypes(include=[np.number])
print("데이터 로드 완료!")
print(f"mRNA: {mrna.shape} | clinical: {clinical.shape} | methyl: {methyl.shape}")

## 임상 데이터 EDA (Clinical Exploratory Data Analysis)
생존 상태, 암 아형, 나이 분포를 한 번에 시각화합니다.
환자 코호트의 기본적인 특성을 파악할 수 있습니다.

In [ ]:
print("\n임상 데이터 분포 분석 중...")

fig, axes = plt.subplots(1, 3, figsize=(16,4))

# 생존 상태 원형 차트
clinical[STATUS_COL].value_counts().plot(kind="pie", autopct="%1.0f%%", ax=axes[0], title="Survival Status")

# 암 아형별 환자 수
sns.countplot(x=SUBTYPE_COL, data=clinical, ax=axes[1])
axes[1].set_title("Subtype Distribution")
axes[1].tick_params(axis="x", rotation=30)

# 나이 분포 히스토그램
clinical[AGE_COL].plot(kind="hist", bins=20, edgecolor="black", ax=axes[2], title="Age Distribution")

plt.tight_layout()
plt.show()

print("임상 데이터 분석 완료!")

## mRNA 발현 데이터 EDA
상위 변이 유전자들의 발현 분포를 다양한 방법으로 시각화합니다.
샘플 간 상관관계와 차원축소를 통한 패턴 분석을 수행합니다.

In [ ]:
print("\nmRNA 발현 데이터 분석 중...")

# 서브플롯 구성 (2행 2열)
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle("mRNA Data Analysis", fontsize=16)

# 1. 바이올린 플롯 (상위 5개 유전자)
print("상위 5개 유전자 발현 분포 분석...")
sns.violinplot(data=mrna_num.iloc[:5].T, ax=axes[0, 0])
axes[0, 0].set_title("Expression Distribution (Top 5 Genes)")
axes[0, 0].tick_params(axis='x', rotation=45)

# 2. 밀도 플롯 (첫 번째 유전자)
g = mrna_num.index[0]
sns.kdeplot(mrna_num.loc[g], fill=True, ax=axes[0, 1])
axes[0, 1].set_title(f"Expression Density: {g}")

# 3. 샘플 간 상관관계 클러스터맵
print("샘플 간 상관관계 분석...")
# clustermap은 별도의 Figure로 생성되므로 subplot에 직접 그리기 어려움 → heatmap으로 대체
sns.heatmap(mrna_num.corr(), cmap="vlag", ax=axes[1, 0])
axes[1, 0].set_title("Sample Correlation Heatmap")

# 4. PCA 차원축소 분석
print("PCA 차원축소 분석...")
X = StandardScaler().fit_transform(mrna_num.T.fillna(0))
pca = PCA(n_components=2).fit(X)
pcs = pca.transform(X)
axes[1, 1].scatter(pcs[:, 0], pcs[:, 1])
axes[1, 1].set_title(f"PCA Plot (PC1 explains {pca.explained_variance_ratio_[0]:.1%})")
axes[1, 1].set_xlabel("PC1")
axes[1, 1].set_ylabel("PC2")

# 전체 레이아웃 조정
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

print("mRNA 발현 데이터 분석 완료!")

### Vibe Coding 미션 A1: mRNA 발현 분석 심화
**미션**: mRNA 발현 데이터 시각화를 개선해보세요!

**도전 과제**:
1. 바이올린 플롯 색상을 'husl' 팔레트로 변경하기
2. 밀도 플롯에 평균과 중앙값 선 추가하기  
3. PCA 산점도에 암 아형별 색상 적용하기

In [ ]:
# 미션 A1 코드를 여기에 작성하세요!


### Vibe Coding 미션 A2: 고급 유전자 발현 분석
**미션**: 특정 변이 유전자들만 선별하여 정교한 클러스터맵을 만들어보세요!

**도전 과제**:
1. 무작위 30개 변이 유전자 선별하기
2. 클러스터맵에 컬러바 라벨과 주석 추가하기
3. 범례 위치 지정하기


In [ ]:
# 미션 A2 코드를 여기에 작성하세요!



## 메틸화 데이터 EDA (Methylation Analysis)
메틸화 베타값의 분포와 상위 변이 프로브들을 분석합니다.
프로브별 메틸화 패턴을 히트맵으로 시각화합니다.

In [ ]:
print("\n메틸화 데이터 분석 중...")

# 숫자형 변환 및 전처리
print("메틸화 데이터 전처리...")
meth = methyl.apply(pd.to_numeric, errors="coerce").groupby(level=0).mean()
min_valid = max(3, int(0.2 * meth.shape[1]))
meth = meth[meth.notna().sum(axis=1) >= min_valid].clip(0, 1)

print(f"전처리 완료: {meth.shape}")

In [ ]:
print("메틸화 데이터 분석 시작...")

# 서브플롯 (1행 2열)
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
fig.suptitle("Methylation Data Analysis", fontsize=16)

# 1. 베타값 분포 분석 (왼쪽)
print("베타값 분포 분석...")
axes[0].hist(meth.iloc[:, 0].dropna().clip(0, 1), bins=40, edgecolor="black", alpha=0.7)
axes[0].set_title("Methylation Beta Value Distribution (First Sample)")
axes[0].set_xlabel("Beta Value")
axes[0].set_ylabel("Count")

# 2. 상위 변이 프로브 선별 및 히트맵 (오른쪽)
print("상위 변이 프로브 선별...")
var = meth.var(axis=1, skipna=True).sort_values(ascending=False)
top = var.head(min(30, len(var))).index
print(f"상위 30개 변이 프로브 선별 완료")

print("메틸화 패턴 히트맵 생성...")
subset = meth.loc[top, meth.columns[:min(30, meth.shape[1])]]
sns.heatmap(
    subset,
    cmap="RdYlBu_r",
    center=0.5,
    cbar_kws={'label': 'Beta Value'},
    ax=axes[1]
)
axes[1].set_title("Top Variable Methylation Probes")
axes[1].set_xlabel("Samples")
axes[1].set_ylabel("Probes")
axes[1].tick_params(axis='x', rotation=45)
axes[1].tick_params(axis='y', rotation=0)

# 전체 레이아웃 정리
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

print("메틸화 데이터 분석 완료!")


### Vibe Coding 미션 B1: 메틸화 분포 시각화 개선
**미션**: 메틸화 베타값 분포를 더 자세히 분석해보세요!

**도전 과제**:
1. 히스토그램 bin 개수를 50개로 설정하기
2. 처음 5개 샘플의 분포를 서브플롯으로 비교하기
3. KDE 곡선을 히스토그램에 추가하기


In [ ]:
# 미션 B1 코드를 여기에 작성하세요!


### Vibe Coding 미션 B2: 프로브별 메틸화 비교 분석
**미션**: 상위 변이 프로브들의 메틸화 패턴을 박스플롯으로 비교해보세요!

**도전 과제**:
1. 상위 10개 변이 프로브 선별하기
2. 각 프로브별 베타값 분포를 박스플롯으로 시각화하기
3. 아웃라이어가 있는 프로브 식별하고 표시하기


In [ ]:
# 미션 B2 코드를 여기에 작성하세요!


## DMP (Differential Methylated Probe) 분석
암 아형간 메틸화 차이가 큰 프로브를 통계적으로 식별합니다.
Welch t-test와 FDR 보정을 통한 다중검정 보정을 수행합니다.
결과를 Volcano plot으로 시각화합니다.

In [ ]:
print("\nDMP (Differential Methylated Probe) 분석 중...")

# 분석 파라미터 설정
diff_thr = 0.10   # |Δβ| 컷오프
fdr_thr  = 0.05   # FDR 컷오프
min_per_group = 5

print(f"분석 파라미터: |Δβ| ≥ {diff_thr}, FDR < {fdr_thr}")

# 공통 샘플 정렬
clinical_sub = clinical[[SAMPLE_COL, SUBTYPE_COL]].dropna()
common = methyl.columns.intersection(clinical_sub[SAMPLE_COL])

methyl_use   = methyl.loc[:, common]
clinical_use = clinical_sub[clinical_sub[SAMPLE_COL].isin(common)]

# 비교할 두 subtype 자동 선택 (샘플 수 상위 2개)
vc = clinical_use[SUBTYPE_COL].value_counts()
subtype_a, subtype_b = vc.index[:2].tolist()
print(f"비교 그룹: {subtype_a} vs {subtype_b}")

# 그룹 샘플 벡터
group_A = clinical_use.loc[clinical_use[SUBTYPE_COL].eq(subtype_a), SAMPLE_COL].tolist()
group_B = clinical_use.loc[clinical_use[SUBTYPE_COL].eq(subtype_b), SAMPLE_COL].tolist()

print(f"그룹 크기: {subtype_a} (n={len(group_A)}), {subtype_b} (n={len(group_B)})")

In [ ]:
# 평균 차이 계산 (Δβ = A - B)
mean_A = methyl_use[group_A].mean(axis=1)
mean_B = methyl_use[group_B].mean(axis=1)
diff   = mean_A - mean_B

# Welch t-test (열=feature가 되도록 전치)
print("통계 검정 수행 중...")
t_stat, p_val = stats.ttest_ind(methyl_use[group_A].T, methyl_use[group_B].T,
                                equal_var=False, nan_policy="omit")
p_val = np.asarray(p_val, float)
p_val = np.where(np.ndim(p_val)==0, np.full(methyl_use.shape[0], np.nan), p_val)

# Benjamini–Hochberg FDR 보정
print("   🔧 다중검정 보정 (FDR) 수행 중...")
order  = np.argsort(p_val)
ranked = p_val[order]
n      = ranked.size
q_tmp  = ranked * n / (np.arange(n) + 1)
q_adj  = np.minimum.accumulate(q_tmp[::-1])[::-1]
q_val  = np.empty_like(q_adj)
q_val[order] = q_adj

# 결과 테이블 생성
dmp = pd.DataFrame({"diff": diff.values, "p": p_val, "q": q_val},
                   index=methyl_use.index).sort_values("q")

print(f"DMP 테이블 생성 완료 (총 {len(dmp)}개 프로브)")

In [ ]:
# Volcano plot 생성
print("Volcano plot 생성 중...")
x = dmp["diff"].to_numpy()
y = -np.log10(np.clip(dmp["p"].to_numpy(), 1e-300, 1.0))

plt.figure(figsize=(10, 7))
plt.scatter(x, y, s=8, alpha=0.6, c='lightblue', edgecolors='none')

# 임계선 추가
plt.axhline(-np.log10(fdr_thr), ls="--", c="red", alpha=0.7, linewidth=2, label=f"FDR {fdr_thr}")
plt.axvline(+diff_thr, ls="--", c="green", alpha=0.7, linewidth=2)
plt.axvline(-diff_thr, ls="--", c="green", alpha=0.7, linewidth=2, label=f"|Δβ| {diff_thr}")

# 유의한 프로브 하이라이트
sig_mask = (dmp["q"] < fdr_thr) & (dmp["diff"].abs() >= diff_thr)
plt.scatter(x[sig_mask], y[sig_mask], s=15, alpha=0.8, c='red', edgecolors='darkred', label='Significant')

plt.title(f"DMP Volcano Plot: {subtype_a} vs {subtype_b}")
plt.xlabel("Mean β difference (A - B)")
plt.ylabel("-log₁₀(p-value)")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 결과 요약
print("DMP 분석 결과:")
print(f"유의한 DMP: {sig_mask.sum()}개 (FDR<{fdr_thr}, |Δβ|≥{diff_thr})")
print(f"상위 5개 hypermethylated 프로브:")
top_hyper = dmp[dmp["diff"] > 0].head(5)
for i, (probe, row) in enumerate(top_hyper.iterrows(), 1):
    print(f"         {i}. {probe}: Δβ={row['diff']:.3f}, FDR={row['q']:.2e}")

print(f"상위 5개 hypomethylated 프로브:")
top_hypo = dmp[dmp["diff"] < 0].head(5)
for i, (probe, row) in enumerate(top_hypo.iterrows(), 1):
    print(f"         {i}. {probe}: Δβ={row['diff']:.3f}, FDR={row['q']:.2e}")

print("DMP 분석 완료!")

### Vibe Coding 미션 B3: DMP 분석 심화
**미션**: DMP 결과에서 상위 유의한 프로브들의 메틸화 패턴을 히트맵으로 시각화해보세요!

**도전 과제**:
1. 상위 20개 유의한 DMP 선별하기
2. 샘플을 Subtype별로 정렬하여 히트맵 생성하기  
3. 히트맵에 그룹 구분선과 컬러바 추가하기


In [ ]:
# 미션 B3 코드를 여기에 작성하세요!
